In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
# CASSANDRA CONFIGURATION
cassandra_host = "cassandra"
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
      .config("spark.cassandra.connection.host", cassandra_host) \
      .config("spark.jars.packages","com.datastax.spark:spark-cassandra-connector-assembly_2.12:3.1.0")\
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.datastax.spark#spark-cassandra-connector-assembly_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8d689389-966d-4cab-9682-4feab449c611;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0 in central
downloading https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector-assembly_2.12/3.1.0/spark-cassandra-connector-assembly_2.12-3.1.0.jar ...
	[SUCCESSFUL ] com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0!spark-cassandra-connector-assembly_2.12.jar (2235ms)
:: resolution report :: resolve 1764ms :: artifacts dl 2238ms
	:: modules in use:
	com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   ar

In [2]:
!pip install -q cassandra-driver


In [3]:
import requests
response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=43.0481&longitude=76.1474&start_date=2023-05-16&end_date=2023-05-30&hourly=temperature_2m,dewpoint_2m,precipitation,rain,snowfall,weathercode,cloudcover,cloudcover_low,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m,windgusts_10m&temperature_unit=fahrenheit')
#response = requests.get('https://api.open-meteo.com/v1/forecast?latitude=43.05&start_date=2023-05-16&end_date=2023-05-30&longitude=-76.15&daily=weathercode,temperature_2m_max,temperature_2m_min,sunrise,sunset,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,windspeed_10m_max,windgusts_10m_max&temperature_unit=fahrenheit&timezone=auto')
data = response.json()


In [4]:
from pyspark.sql.functions import col,explode
import pandas as pd
df = pd.json_normalize(data)

#save data to json file and csv
df.to_json("project_weather_data_meteo.json2", orient="records")
df.to_csv("project_weather_data_meteo.csv2",index=False,header=True)



In [5]:
import csv
#try to rename the column from the json
#hourly_data = data['daily']

#variables = hourly_data.keys()

# Write the flattened data to a CSV file
#with open('project_weather_data_meteo.csv2', 'w', newline='') as file:
   # writer = csv.writer(file)

    # Write the header row with variable names
    writer.writerow(['Date', 'Time'] + list(variables))

    # Iterate over each hour
   # for i in range(len(hourly_data['time'])):
        row = []

        # Extract the date and time
    #    date, time = hourly_data['time'][i].split('T')
     #   row.extend([date, time])

        # Extract the values for each variable
     #   for variable in variables:
      #      value = hourly_data[variable][i]
       #     row.append(value)

     #   writer.writerow(row)
#

KeyError: 'daily'

In [6]:
#open csv 
file_path = 'file:///home/jovyan/datasets/project_weather_data_meteo.json'
weather = spark.read.json(file_path)
weather.printSchema()
#save as dataframe and see the schema 


root
 |-- elevation: double (nullable = true)
 |-- generationtime_ms: double (nullable = true)
 |-- hourly: struct (nullable = true)
 |    |-- cloudcover: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- cloudcover_low: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- dewpoint_2m: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- precipitation: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- rain: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- snowfall: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- temperature_2m: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- time: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- weathercode: array (nullable = true)
 |    |    |-- element: long (cont

In [ ]:
weathertest = weather.select(weather.hourly.time)
#weathertest = weather.select(weather.hourly_units.time)
#df = weather.select(explode("hourly_units.time").alias("time")).select("time.*")


In [7]:
file_weather =  "/home/jovyan/datasets/project_weather_data_meteo.csv"

df_weather= pd.read_csv(file_weather)
df_weather = df_weather.rename(columns={'Time': 'hour'})

df_spark = spark.createDataFrame(df_weather)


In [8]:
df_spark.printSchema()

root
 |-- Date: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- time: string (nullable = true)
 |-- temperature_2m: double (nullable = true)
 |-- dewpoint_2m: double (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- rain: double (nullable = true)
 |-- snowfall: double (nullable = true)
 |-- weathercode: long (nullable = true)
 |-- cloudcover: long (nullable = true)
 |-- cloudcover_low: long (nullable = true)
 |-- windspeed_10m: double (nullable = true)
 |-- windspeed_100m: double (nullable = true)
 |-- winddirection_10m: long (nullable = true)
 |-- winddirection_100m: long (nullable = true)
 |-- windgusts_10m: double (nullable = true)



In [9]:

# Save the data to Cassandra
create_table_sql =  '''
CREATE TABLE IF NOT EXISTS project_weather_data.weather_data_table (
    date date, 
    hour time,
    time timestamp,
    temperature_2m float,
    dewpoint_2m float,
    precipitation float,
    rain float,
    snowfall float,
    weathercode float,
    cloudcover float,
    cloudcover_low float,
    windspeed_10m float,
    windspeed_100m float,
    winddirection_10m float,
    winddirection_100m float,
    windgusts_10m float,
    PRIMARY KEY (time)



  );
  '''
from cassandra.cluster import Cluster
with Cluster([cassandra_host]) as cluster:
    session = cluster.connect()
    session.execute(create_table_sql)
print(create_table_sql)


CREATE TABLE IF NOT EXISTS project_weather_data.weather_data_table (
    date date, 
    hour time,
    time timestamp,
    temperature_2m float,
    dewpoint_2m float,
    precipitation float,
    rain float,
    snowfall float,
    weathercode float,
    cloudcover float,
    cloudcover_low float,
    windspeed_10m float,
    windspeed_100m float,
    winddirection_10m float,
    winddirection_100m float,
    windgusts_10m float,
    PRIMARY KEY (time)



  );
  


In [10]:
weather_2 = df_spark.toDF("date",
"hour",
"time",
"temperature_2m",
"dewpoint_2m",
"precipitation",
"rain",
"snowfall",
"weathercode",
"cloudcover",
"cloudcover_low",
"windspeed_10m",
"windspeed_100m",
"winddirection_10m",
"winddirection_100m",
"windgusts_10m");


In [11]:
weather_2.write.format("org.apache.spark.sql.cassandra")\
    .mode("Append")\
    .option("table", "weather_data_table")\
    .option("keyspace", "project_weather_data")\
    .save()

In [13]:
weather_3 =spark.read.format("org.apache.spark.sql.cassandra")\
    .option("table", "weather_data_table")\
    .option("keyspace","project_weather_data")\
    .load()

weather_3.count()

360

In [15]:
 weather_3.createOrReplaceTempView("weather")
query =  '''
 SELECT time, temperature_2m, precipitation, rain, snowfall, weathercode FROM weather_data_table;
 '''
spark.sql(query).explain()


== Physical Plan ==
*(1) Project [time#123, temperature_2m#132, precipitation#129, rain#130, snowfall#131, weathercode#133]
+- BatchScan[time#123, precipitation#129, rain#130, snowfall#131, temperature_2m#132, weathercode#133] Cassandra Scan: project_weather_data.weather_data_table
 - Cassandra Filters: []
 - Requested Columns: [time,precipitation,rain,snowfall,temperature_2m,weathercode]




In [25]:
query =  '''
 SELECT time, temperature_2m, precipitation, rain, snowfall, weathercode from weather_data_table  WHERE temperature_2m  > '60'
 '''
spark.sql(query).explain()


== Physical Plan ==
*(1) Project [time#123, temperature_2m#132, precipitation#129, rain#130, snowfall#131, weathercode#133]
+- *(1) Filter (temperature_2m#132 > 60.0)
   +- BatchScan[time#123, precipitation#129, rain#130, snowfall#131, temperature_2m#132, weathercode#133] Cassandra Scan: project_weather_data.weather_data_table
 - Cassandra Filters: []
 - Requested Columns: [time,precipitation,rain,snowfall,temperature_2m,weathercode]


